# This notebook for testing code
#### The final version will be saved as indivudual python files that can be executed on the terminal or called by another python program

In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser

In [ ]:
# Initialize PyMongo to work with MongoDBs
client = pymongo.MongoClient('mongodb://localhost:27017')

In [ ]:
# Define database and collection
db = client.mars
collection = db.marspage

In [39]:
## PART 1 Mars News
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
response = requests.get(url, timeout=5)

In [40]:
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'html.parser')

In [45]:
header = soup.find('div', class_='content_title')
body = soup.find('div', class_='rollover_description_inner')
print(header.text)
print(body.text)
news_dict = {}
news_dict['article_title'] = header.text.strip()
news_dict['article_content'] = body.text.strip()
print(news_dict)



NASA's InSight Places First Instrument on Mars



In deploying its first instrument onto the surface of Mars, the lander completes a major mission milestone.

{'article_title': "NASA's InSight Places First Instrument on Mars", 'article_content': 'In deploying its first instrument onto the surface of Mars, the lander completes a major mission milestone.'}


In [ ]:
### PART 2 FEATURED IMAGE

In [2]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
featured_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(featured_url)
browser.click_link_by_partial_text('FULL IMAGE')

In [4]:
featured_html = browser.html
featured_soup = BeautifulSoup(featured_html, 'html.parser')
url_suffix = featured_soup.find('img', class_='fancybox-image').attrs['src']
featured_img_url = browser.url + url_suffix
print(f'Featured img url: {featured_img_url}')

Featured img url: https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/mediumsize/PIA16153_ip.jpg


In [5]:
type(featured_soup.find('img', class_='fancybox-image'))

bs4.element.Tag

In [50]:
featured_dict = {}
featured_dict['title'] = 'Featured_Image'
featured_dict['img_url'] = featured_img_url
print(featured_dict)

{'title': 'Featured_Image', 'img_url': 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/mediumsize/PIA17845_ip.jpg'}


In [ ]:
### PART 3 TWITTER SCRAPING
twitter_url='https://twitter.com/marswxreport?lang=en'

In [ ]:
twit_resp = requests.get(twitter_url, timeout=5)

In [ ]:
twitter_soup = BeautifulSoup(twit_resp.text, 'html.parser')

In [ ]:
tweets = twitter_soup.find_all('p', class_='TweetTextSize')
for tweet in tweets:
    if tweet.text[:3] == 'Sol':
        weather_dict = {}
        weather_dict['weather'] = tweet.text
        print(tweet.text)
        break
    else: next

In [ ]:
## PART 4 MARS FACTS
fact_url = 'https://space-facts.com/mars/'
fact_resp = requests.get(fact_url, timeout=5)

In [ ]:
fact_soup = BeautifulSoup(fact_resp.text, 'html.parser')
facts = fact_soup.find_all('tr')
for fact in facts:
    print(fact.text)

In [ ]:
#this version prints all the facts into one dictionary
fact_dict = {}
for fact in facts:
    description, value = fact.text.strip().split(':')
    fact_dict[description] = value    
print(fact_dict)

In [ ]:
#this version puts the facts into a list of individual dictionaries
fact_list = []
for fact in facts:
    fact_dict = {}
    description, value = fact.text.strip().split(':')
    fact_dict[description] = value
    fact_list.append(fact_dict)
    ##insert to Mongo DB
print(fact_list)

In [ ]:
###Part 5 Mars Hemispheres

In [35]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [36]:
hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemi_url)

In [38]:
hemispheres = ['Cerberus','Schiaparelli','Syrtis','Valles']
hemi_list = []
for hemi in hemispheres:
    hemi_dict = {}
    browser.click_link_by_partial_text(hemi)
    hemi_soup = BeautifulSoup(browser.html, 'html.parser')
    hemi_title = hemi_soup.find('h2', class_='title')
    hemi_title = hemi_title.text.split(' Enhanced')[0]
    print(hemi_title)
    dl_tag = hemi_soup.find('div', class_='downloads')
    hemi_img = dl_tag.find('a').attrs['href']
    print(hemi_img)
    hemi_dict['title'] = hemi_title
    hemi_dict['img_url'] = hemi_img
    hemi_list.append(hemi_dict)
    browser.back()
print(hemi_list)

Cerberus Hemisphere
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
Schiaparelli Hemisphere
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
Syrtis Major Hemisphere
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
Valles Marineris Hemisphere
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
[{'title': 'Cerberus Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'http://astropedia.astrogeology

In [ ]:
### LEFTOVERS FROM HOCKEY ASSIGNMENT
headers = soup.find_all('li', class_='mixed-feed__item--article')
for h in headers:
    try:
        title = h.find('h4').text
        sub = h.find('h5').text
        logo = h.find('img', class_='logo').attrs['src']
        print(title)
        print(' - '+sub)
        print(f"https://{logo}")
        print('-----------------')
        
        article = {
            'title': title,
            'subtitle': sub,
            'team_logo': logo 
        }
              
        collection.insert_one(article)
              
    except Exception as e: 
        print(e)

In [ ]:
# Display the MongoDB records created above
articles = db.articles.find()
for article in articles:
    print(article)